In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import tensorflow as tf
import transformers
from transformers import TFAutoModel, AutoTokenizer

In [3]:
raw = pd.read_csv('/content/Suicide_Detection.csv')

In [4]:
len(raw)

232074

In [12]:
raw['contains_keyword'] = raw['text'].str.contains(pattern, case=False, na=False)


,Unnamed: 0,text,class,contains_keyword
20,30,"I am ending my life today, goodbye everyone.I ...",suicide,True
38,51,revenge suicideDoes the thought ever cross you...,suicide,True
91,135,How much time do people spend feeling aloneI h...,suicide,True
99,147,2929\n\n\nThat’s the oldest I’ve ever been. It...,suicide,True
202,294,Never EndingI’ve been going through shit this ...,suicide,True
...,...,...,...,...
231508,347257,"I want to die, but I don't want to hurt the pe...",suicide,True
231645,347457,It’s like I can’t even be happy anymoreEveryth...,suicide,True
231650,347464,"I want to die, but I'm afraid it'll kill someo...",suicide,True
231872,347811,I've been like this for a long time and I want...,suicide,True


In [13]:
df_keywords = raw[raw['contains_keyword']==True]

In [20]:
df = raw.sample(20000 , random_state=29)

In [21]:
df = pd.concat([df, df_keywords], ignore_index=True)
df.head()

,Unnamed: 0,text,class,contains_keyword
0,319568,Nobody cares...Got to see / talk to my mom for...,suicide,False
1,321119,Lost and confusedSo this whole bout of depress...,suicide,False
2,204141,My life is falling apart and I don't know want...,suicide,False
3,289022,Mental BarrierHonestly I've stopped trying to ...,suicide,False
4,298459,I went ghost to focus on myself for a week and...,non-suicide,False


In [22]:
df= df.drop('Unnamed: 0',axis=1)
df= df.drop('contains_keyword',axis=1)

In [23]:
df['class'].replace({'suicide' : 1 , 'non-suicide' : 0}, inplace=True)

In [24]:
df

,text,class
0,Nobody cares...Got to see / talk to my mom for...,1
1,Lost and confusedSo this whole bout of depress...,1
2,My life is falling apart and I don't know want...,1
3,Mental BarrierHonestly I've stopped trying to ...,1
4,I went ghost to focus on myself for a week and...,0
...,...,...
22206,"I want to die, but I don't want to hurt the pe...",1
22207,It’s like I can’t even be happy anymoreEveryth...,1
22208,"I want to die, but I'm afraid it'll kill someo...",1
22209,I've been like this for a long time and I want...,1


In [9]:
keywords = ['Kms', 'kys', 'self-harm', 'ending my life']
pattern = '|'.join(keywords)
df['contains_keyword'] = df['text'].str.contains(pattern, case=False, na=False)

In [11]:
df[df['contains_keyword']==True]

,text,class,contains_keyword
94458,I want to and Im okay with it.I think im going...,1,True
9396,im losing all that matters an now im going to ...,1,True
93978,I am next to our bridge and I have a knife + p...,1,True
104569,UPDATE: I'm gonn kms on christmasSo hey again ...,1,True
132940,My last 12 hoursWhat should I do? As you can g...,1,True
...,...,...,...
202626,"i legit wanna kmsi'm literally bipolar type 2,...",1,True
188605,Keeping it inI can't help but not tell anyone ...,1,True
198712,"Need Advice Please!Hi everyone,\nSomeone I kno...",1,True
141215,what's the pointIdk I realise I should be grat...,1,True


In [81]:
df.head()

,text,class
0,Nobody cares...Got to see / talk to my mom for...,1
1,Lost and confusedSo this whole bout of depress...,1
2,My life is falling apart and I don't know want...,1
3,Mental BarrierHonestly I've stopped trying to ...,1
4,I went ghost to focus on myself for a week and...,0


In [25]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [26]:
encoded = tokenizer(df['text'].tolist(), truncation=True, padding=True)
encoded_dataset = tf.data.Dataset.from_tensor_slices((encoded, df['class'].tolist()))

In [27]:
encoded_dataset

<_TensorSliceDataset element_spec=({'input_ids': TensorSpec(shape=(512,), dtype=tf.int32, name=None), 'token_type_ids': TensorSpec(shape=(512,), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(512,), dtype=tf.int32, name=None)}, TensorSpec(shape=(), dtype=tf.int32, name=None))>

In [28]:
BATCH_SIZE = 2

def order(inp, label):

    return {
        'input_ids': inp['input_ids'],
        'attention_mask': inp['attention_mask'],
        'token_type_ids': inp['token_type_ids']
    }, label



train_size = int(0.8 * len(encoded_dataset))
val_size = int(0.1 * len(encoded_dataset))

train_dataset = encoded_dataset.take(train_size).map(order)
val_dataset = encoded_dataset.skip(train_size).take(val_size).map(order)
test_dataset = encoded_dataset.skip(train_size + val_size).map(order)


train_dataset = train_dataset.map(order)
val_dataset = val_dataset.map(order)
test_dataset = test_dataset.map(order)


train_dataset = train_dataset.batch(BATCH_SIZE).shuffle(1000)
val_dataset = val_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [29]:
text, label = next(iter(train_dataset))
print(text, '\n\n', label)

{'input_ids': <tf.Tensor: shape=(2, 512), dtype=int32, numpy=
array([[  101,  2298, 10047, ...,     0,     0,     0],
       [  101,  6612,  2135, ...,  2424,  2870,   102]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 512), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 1, 1, 1]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(2, 512), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>} 

 tf.Tensor([0 1], shape=(2,), dtype=int32)


In [30]:
bert_model = TFAutoModel.from_pretrained("bert-base-uncased")

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [31]:
class BERTForClassification(tf.keras.Model):

    def __init__(self, bert_model):
        super().__init__()
        self.bert = bert_model
        self.fc = tf.keras.layers.Dense(1, activation='sigmoid')

    def call(self, inputs):
        x = self.bert(inputs)[1]
        return self.fc(x)

    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'bert_model': self.bert,
            'num_classes': self.fc.units,
        })
        return config


In [32]:
model = BERTForClassification(bert_model)

In [33]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=['accuracy']
)


In [34]:
history = model.fit(
    train_dataset,
    epochs=2,
    validation_data=val_dataset
)

Epoch 1/2
8884/8884 [==============================] - 2654s 293ms/step - loss: 0.1425 - accuracy: 0.9451 - val_loss: 0.0914 - val_accuracy: 0.9667
Epoch 2/2
8884/8884 [==============================] - 2594s 292ms/step - loss: 0.0582 - accuracy: 0.9800 - val_loss: 0.1474 - val_accuracy: 0.9685


In [35]:
model.evaluate(test_dataset)

1111/1111 [==============================] - 113s 102ms/step - loss: 0.2261 - accuracy: 0.9577


[0.22609302401542664, 0.9576957821846008]

In [ ]:
y_true = []
for batch in test_dataset:
    x, y = batch
    y_true.extend(y.numpy())

y_pred_prob = model.predict(test_dataset)
y_pred = np.round(y_pred_prob).astype(int)

cm = confusion_matrix(y_true, y_pred)
print(cm)

In [88]:
test_m = ["I cut myself", "I wanna kill myself", "I am going to cut my bread"]

inputs = tokenizer(test_m ,padding=True, truncation=True,
                  return_tensors='tf')
model.call(inputs)

<tf.Tensor: shape=(3, 1), dtype=float32, numpy=
array([[9.242630e-01],
       [9.989396e-01],
       [9.893739e-05]], dtype=float32)>

In [75]:
model.save('/content/drive/MyDrive')


In [76]:
import tensorflow as tf

# Make sure to use the correct path where your MyModel directory is located in your Google Drive
new_model = tf.keras.models.load_model('/content/drive/MyDrive/MyModel')


In [78]:
test_m = ["Today I felt good in the morning, everything was good, but in the evening, it rained, and as a result, I got stuck in the traffic; my life sucks",
           "Today, I felt good in the morning; everything was good, but in the evening, it rained, and as a result, I got stuck in traffic. My life sucks; I should end it; I should kill myself.",
           "i cut myself several times and it s already scarred honestly if i go through all of that i m sure a little bit of work would nt hurt but if i was about to kill myself several times in quarantine and lost motivation for everything but still did my work",
           "I am going to cut my bread",
           "I wanna kms",
          "She just killed herself",
           "Why dont you kys",
           "This video is killing me",
           "while now now i also can cut myself off with a smile set on board my face",
          "I cut a piece of bread"]

inputs = tokenizer(test_m, padding='max_length', truncation=True, max_length=512, return_tensors='tf')

new_model.call(inputs)

<tf.Tensor: shape=(10, 1), dtype=float32, numpy=
array([[7.0992403e-04],
       [9.9993181e-01],
       [7.9414022e-01],
       [9.8937300e-05],
       [9.1173402e-05],
       [6.0861990e-02],
       [3.9682312e-05],
       [6.8748742e-04],
       [1.5628006e-01],
       [9.8359138e-05]], dtype=float32)>